In [ ]:
from osgeo import gdal
import glob

In [ ]:
#CAUTION: the script will overwrite the .tif files in this folder

#Only a x,y translation is done. No rotation nor scaling.

# Folder with the images to be shifted:

#folder = '/media/nmorandeira/T7/Img_preprocesadas/SAOCOM/SAOCOM_StripMap_QP/polariz/Asc_S8/shifted'
#folder = '/media/nmorandeira/T7/Img_preprocesadas/SAOCOM/SAOCOM_StripMap_QP/polariz/Asc_S9/shifted'
folder = '/home/nmorandeira/Documents/ExtraccionSAR/SAOCOM shift prueba/shifted'


img_list = glob.glob(folder+'/*.tif')
#if the scenes are organized in subfolfers:
#img_list = glob.glob(folder+'/*/*.tif')

print (img_list)

In [ ]:
# pixel shifts per SAOCOM beam

# Descending S7 - no shift needed

# Ascending S8
pix_x_S8 = 10
pix_y_S8 = 0

# Ascending S9
pix_x_S9 = 10 
pix_y_S9 = 0 

# Needs improvement: choose pixel shifts according to beam (folder stringr detection)
# manual pixel choice:
pix_x = pix_x_S8
pix_y = pix_y_S8

#pix_x = pix_x_S9
#pix_y = pix_y_S9

In [ ]:
# open dataset with update permission - Caution: the coordinates will be updated (no backup version of the original tiff is saved)

for scene in img_list:
    ds = gdal.Open(scene, gdal.GA_Update)
    # get the geotransform as a tuple of 6
    gt = ds.GetGeoTransform()
    
# unpack geotransform into variables
    x_tl, x_res, dx_dy, y_tl, dy_dx, y_res = gt
    print(gt)
    
    # compute shift of n pixel +RIGHT in X direction
    # x_res (resolution in metres or degrees, depending on the projection) is read by gdal
    
    shift_x = pix_x * x_res 

    # compute shift of n pixels +DOWN in Y direction (Southern Hemisphere) 
    #y_res (resolution in metres or degrees, depending on the projection) is read by gdal
    shift_y = pix_y * y_res

    print(shift_x, shift_y)
    
    # make new geotransform
    gt_update = (x_tl + shift_x, x_res, dx_dy, y_tl + shift_y, dy_dx, y_res)
    # assign new geotransform to raster
    ds.SetGeoTransform(gt_update)
    # ensure changes are committed
    ds.FlushCache()
    ds = None